## Preprocessing

In [76]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [77]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN", "NAME"], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [78]:
# Determine the number of unique values in each column.
# Code taken from StackOverflow: https://stackoverflow.com/questions/27241253/print-the-unique-values-in-every-column-in-a-pandas-dataframe
application_df.astype("object").describe(include="all").loc["unique", :]

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
Name: unique, dtype: object

In [79]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_counts = application_df["APPLICATION_TYPE"].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [80]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_dict = application_counts.to_dict()
cutoff_value = 500
application_types_to_replace = [key for key in application_dict if application_dict.get(key) < cutoff_value]

# Replace in dataframe
for app in application_types_to_replace:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app,"Other")

# Check to make sure replacement was successful
application_df["APPLICATION_TYPE"].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [81]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [82]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# lambda function taken from StackOverflow: https://stackoverflow.com/questions/22320356/pandas-get-values-from-column-that-appear-more-than-x-times
classification_counts.loc[lambda x: x>1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [83]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_dict = classification_counts.to_dict()
cutoff_value = 1000
classifications_to_replace = [key for key in classification_dict if classification_dict.get(key) < cutoff_value]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [84]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_copy = application_df.copy().drop(["ASK_AMT", "IS_SUCCESSFUL"], axis=1)
converted_df = pd.concat([pd.get_dummies(application_df_copy[col], dtype=int) for col in application_df_copy.columns], axis=1)
converted_df = pd.concat([converted_df, application_df["ASK_AMT"], application_df["IS_SUCCESSFUL"]], axis=1)
converted_df.columns = converted_df.columns.astype(str)
converted_df.head()

,Other,T10,T19,T3,T4,T5,T6,T7,T8,CompanySponsored,...,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M,N,Y,ASK_AMT,IS_SUCCESSFUL
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,5000,1
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,108590,1
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,5000,0
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,6692,1
4,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,142590,1


In [85]:
# Split our preprocessed data into our features and target arrays
X = converted_df.drop("IS_SUCCESSFUL", axis=1)
y = converted_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [86]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [87]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=44))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

C:\Users\matth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_19 (Dense)                │ (None, 80)             │         3,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,061 (23.68 KB)

 Trainable params: 6,061 (23.68 KB)

 Non-trainable params: 0 (0.00 B)

In [88]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [89]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 626us/step - accuracy: 0.7054 - loss: 0.5903
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 627us/step - accuracy: 0.7338 - loss: 0.5526
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 619us/step - accuracy: 0.7296 - loss: 0.5525
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 614us/step - accuracy: 0.7317 - loss: 0.5478
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 617us/step - accuracy: 0.7346 - loss: 0.5463
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 615us/step - accuracy: 0.7323 - loss: 0.5438
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 617us/step - accuracy: 0.7337 - loss: 0.5454
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 610us/step - accuracy: 0.7307 - loss: 0.5461
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 614us/step - accuracy: 0.7403 - loss: 0.5417
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 617us/step - accuracy: 0.7351 - loss: 0.5434
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 613us/step - accuracy: 0.7386 - loss: 0.5395
Epoch 12/100
804/80

In [90]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 888us/step - accuracy: 0.7240 - loss: 0.5924
Loss: 0.5923685431480408, Accuracy: 0.7239649891853333


In [92]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")